In [1]:
fp = r"/Users/gugujixiao/workspace/project/HWMemory/Code/newCFG/hw-memory/benchmarks/old_example/func.asm"

In [2]:
from newCFG.cfg import proc_identify
from newCFG.isa import Instruction, AddrMode
from typing import Tuple
from newCFG.read_asm import AsmFileReader, StatementType

reader =AsmFileReader(fp)

statements = list()
temp_stat = list()
for s in reader.statements:
    s: Tuple[StatementType, tuple]
    if s[0] == StatementType.Instruction:
        statements.append((s[0], Instruction(s[1])))
        temp_stat.append(Instruction(s[1]))
    elif s[0] == StatementType.SubProcedure:
        statements.append(s)

#for i in temp_stat:
#    booltemp = i.is_ls
#    if booltemp:
#        if i.ls_addr_mode == AddrMode.ImmeAft:
#            print(i.ls_reg_traget)

procs = proc_identify(statements)
for proc in procs:
    print(proc.name, proc.beg_addr.hex_str())
    if False and proc.name == 'main':
        for inst in proc.instruction:
            print(inst.addr.hex_str(), inst.opcode, inst.name, inst.operands, inst.branch_info)
        print()

('40049c', 'a9bf7bfd', 'stp', None, 'x29, x30, [sp, #-16]!')
('40049c', 'a9bf7bfd', 'stp', None, 'x29, x30, [sp, #-16]!')
('4004a8', 'a8c17bfd', 'ldr', None, 'x19, [sp, #-16]!')
('4004a8', 'a8c17bfd', 'ldr', None, 'x19, [sp, #-16]!')
('4004b0', 'a9bf7bf0', 'stp', None, 'x16, x30, [sp, #-16]!')
('4004b0', 'a9bf7bf0', 'stp', None, 'x16, x30, [sp, #-16]!')
('4004b8', 'f947fe11', 'ldr', None, 'x17, [x16, #4088]')
('4004b8', 'f947fe11', 'ldr', None, 'x17, [x16, #4088]')
('4004d4', 'f9400211', 'ldr', None, 'x17, [x16]')
('4004d4', 'f9400211', 'ldr', None, 'x17, [x16]')
('4004e4', 'f9400611', 'ldr', None, 'x17, [x16, #8]')
('4004e4', 'f9400611', 'ldr', None, 'x17, [x16, #8]')
('4004f4', 'f9400a11', 'ldr', None, 'x17, [x16, #16]')
('4004f4', 'f9400a11', 'ldr', None, 'x17, [x16, #16]')
('400510', 'f94003e1', 'ldr', None, 'x1, [sp]')
('400510', 'f94003e1', 'ldr', None, 'x1, [sp]')
('400548', 'f947ec00', 'ldr', None, 'x0, [x0, #4056]')
('400548', 'f947ec00', 'ldr', None, 'x0, [x0, #4056]')
('4005

In [3]:
from graphviz import Digraph
from newCFG.cfg import draw_proc, find_cycle, has_cycle, proc_draw_edges

proc_draw_edges(procs)

is_cycle = has_cycle(procs)
if is_cycle:
    c = [p.name for p in find_cycle(procs)]
    raise RuntimeError("Loop between procedures is not allowed: {}.".format(c))

g: Digraph = draw_proc(procs)
g.render(filename='procedures', directory='./output', format='svg')

'output/procedures.svg'

In [4]:
from newCFG.cfg import CallGraph

call_graph = CallGraph(procs)

g = call_graph.draw_graph()
g.render(filename='call_graph', directory='./output', format='svg')

'output/call_graph.svg'

In [5]:
from newCFG.cfg import TCfg
import random

tcfg = TCfg(call_graph)
tcfg.build_tcfg()

tcfg_nodes = tcfg.all_nodes
tcfg_edges = tcfg.edges

g = tcfg.draw_graph()
g.render(filename='tcfg', directory='./output', format='svg')

#for node in tcfg_nodes:
#    node.set_rw_condition()
#    #print(node.name, node.base_proc.name, node.inst_range)
#    print(node.name, node.base_proc.name)
#    print("出边")
#    for i in node.outgoing_edge:
#        print(i.dst.name)
#    print("入边")
#    for i in node.incoming_edge:
#        print(i.src.name)
#
#    print()

tcfg.build_loop_hrchy()
for l in tcfg.loops:
    #print("{} (in {} children {}): {}->{} {}".format(l.name, l.father.name if l.father is not None else "None", [sub_l.name for sub_l in l.children], l.back_edge.src.name, l.back_edge.dst.name, [n.name for n in l.all_nodes]))
    #print(l.back_edge.src.name,l.back_edge.dst.name)
    l.back_edge.is_backEdge = True

#for n in tcfg.all_nodes:
#    temp = n.inside_loop.name if n.inside_loop is not None else 'None'
#    print("{} {}".format(n.name, temp))

for n in tcfg_nodes:
    n.set_rw_data()

temp_bool = True

while temp_bool:
    no_end = False

    for n in tcfg_nodes:
        n.set_rw_value()
        if not n.no_out:
            no_end = True
        temp_bool = no_end

for e in tcfg_edges:
    if e.is_backEdge:
        e.loop_value = random.randint(0,10)
        e.edge_value = 2

for e in tcfg_edges:
    print(e.src.name,e.dst.name,e.edge_value)


#for n in tcfg_nodes:
#    print(n.name,"n的值",n.node_value)
        

#for e in tcfg_edges:
#    print(e.src.name,e.dst.name,e.is_backEdge)




n0 n2 1.0
n5 n1 1.0
n2 n4 0.5
n2 n3 0.5
n3 n5 0.5
n4 n5 0.5


In [6]:
from newCFG.cfg import TCfg
from newCFG.loadstore import LSProc

lsproc = LSProc(tcfg_nodes)

temp = lsproc.ls_table

for i in temp:
    print(i.node.name,i.ins.tokens,i.is_find,i.reg_target,i.final_addr)

n0 ('400640', 'b9001fe0', 'str', None, 'w0, [sp, #28]') True sp 0
n0 ('400644', 'b9401fe0', 'ldr', None, 'w0, [x33, #28]') True sp 0
n2 ('400608', 'b9000fe0', 'str', None, 'w0, [sp, #12]') True sp 0
n2 ('400614', 'b9400fe0', 'ldr', None, 'w0, [x33, #12]') True 41f000 4321292
n3 ('400620', 'b9400fe0', 'ldr', None, 'w0, [sp, #12]') True sp 0
